In [ ]:
from cs336_basics.bpe import *

# 1. 训练 BPE
input_path = "/mnt/e/bpe_data/lfs-data/TinyStoriesV2-GPT4-valid.txt"
special_tokens = ["<|endoftext|>"]
vocab_size = 10000

vocab, merges = bpe(
    input_path=input_path,
    vocab_size=vocab_size,
    special_tokens=special_tokens,
)

# 打印最长 token 统计（排除特殊 token）
special_bytes = [t.encode('utf-8') for t in special_tokens]
longest = max(
    ((tid, b) for tid, b in vocab.items() if b not in special_bytes),
    key=lambda x: len(x[1]),
    default=(None, b"")
)
print(f"Longest token: ID {longest[0]}, content: {longest[1]}, length: {len(longest[1])}")

# 2. 保存词汇表 (Vocab)
# 推荐格式：每行一个 JSON 或简单的 ID 分隔，以处理换行符 token
vocab_file = "/home/smingtao01/Download/CS336/llm-from-scratch-assignment1-basics/cs336_basics/tinyStory_vocab.txt"
with open(vocab_file, 'w', encoding="utf-8") as f1:
    for token_id, token_bytes in sorted(vocab.items()):
        # 使用 repr 确保换行符 \n 不会破坏文件行数结构
        # 或者使用 latin-1 解码以保持原始字节的可逆性
        token_repr = token_bytes.decode('utf-8', errors='replace')
        f1.write(f"{token_id}\t{token_repr}\n")

# 3. 保存合并规则 (Merges) - 关键部分
# 采用 GPT-2 标准格式：每行两个由空格分隔的 Token
merges_file = "/home/smingtao01/Download/CS336/llm-from-scratch-assignment1-basics/cs336_basics/tinyStory_merges.txt"
with open(merges_file, "w", encoding='utf-8') as f2:
    # BPE 合并是有序的，必须确保 merges 是 list 或按顺序提取
    # 如果你的 bpe 函数返回的是 dict，请确保它是已排序的
    items = merges if isinstance(merges, list) else merges.keys()
    
    for pair in items:
        # pair 预期是一个 tuple: (bytes, bytes)
        # 使用 'latin-1' 解码可以将 0-255 字节安全地转为字符，不会因不是 UTF-8 而崩溃
        p0 = pair[0].decode('latin-1')
        p1 = pair[1].decode('latin-1')
        f2.write(f"{p0} {p1}\n")

print(f"Vocab size: {len(vocab)}")
print(f"Merges count: {len(merges)}")
